In [2]:
from datetime import datetime

In [3]:
datetime.now().strftime("%Y%m%d-%H%M%S")

'20200702-232026'

/home/vishal/anaconda3/envs/uw_model_play2/bin/python

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [121]:
%reload_ext autoreload
%autoreload 2

In [2]:
%load_ext blackcellmagic

In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import pymysql
import pymssql
import sys

In [4]:
sys.path.insert(0, '/home/shared/utils')

In [5]:
from db_utils import *
import kpi_count

In [6]:
iloans_conn = get_iloans_conn()
predicon_conn = get_predicon_db_conn()
bank_app_conn = get_bank_app_conn()

bank_app - loanid, entered_date, bv decisions

predicon_model - loan_id, entered_date, count of scored applicants, #positive

loans - loanid, lead_time_added, #lender approved

## In a Nutshell

### Extracting Data

In [130]:
bank_app = kpi_count.extract_bankapp(bank_app_conn)

In [131]:
model = kpi_count.extract_model(predicon_conn)

In [132]:
funded_loans = kpi_count.extract_funded_loans(iloans_conn)

### Modifying Data

In [133]:
md_bnk = kpi_count.modify_bankapp_db(bank_app)

In [134]:
md_mod = kpi_count.modify_model_db(model)

In [135]:
md_fl = kpi_count.modify_fundedloans_db(funded_loans)

### Merging all the modified datasets and generating model kpi metrics

In [138]:
kpi_func = kpi_count.get_kpi(md_bnk, md_mod, md_fl)

In [139]:
kpi_func

,entered_date,manual_decision,#positive_scored,#model_scored,isLenderApproved
0,2020-06-09,476,76.0,180.0,63
1,2020-06-10,347,114.0,277.0,55
2,2020-06-11,398,127.0,324.0,70
3,2020-06-12,391,132.0,329.0,63
4,2020-06-13,192,48.0,146.0,24
5,2020-06-15,302,94.0,268.0,63
6,2020-06-16,449,124.0,374.0,71
7,2020-06-17,399,109.0,329.0,69
8,2020-06-18,437,126.0,343.0,74
9,2020-06-19,418,108.0,340.0,62
